# Numerical Error

The purpose of this notebook is to illustrate the numerical error which comes from setting $\kappa = 0$.

In [1]:
import numpy as np                  # defines matrix structures
from qpsolvers import solve_qp      # used for quadratic optimization
import gurobipy as gp               # Gurobi optimization interface (1)
from gurobipy import GRB            # Gurobi optimization interface (2)

Utility functions and output settings used in this notebook are defined in the two cells below.

In [2]:
# want to round rather than truncate when printing
np.set_printoptions(threshold=np.inf)

# only show numpy output to five decimal places
np.set_printoptions(formatter={'float_kind':"{:.5f}".format})

### Toy Example ($n = 3$)

Lets see how this works in an example. We will start by looking how this problem might be solving using Python's [qpsolvers](https://qpsolvers.github.io/qpsolvers/index.html) library. Consider the problem where
$$
    \mu = \begin{bmatrix} 1 \\ 5 \\ 2 \end{bmatrix},\quad
    \Sigma = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 5 & 0 \\ 0 & 0 & 3 \end{bmatrix}, \quad
    \mathcal{S} = \lbrace 1 \rbrace, \quad
    \mathcal{D} = \lbrace 2, 3 \rbrace, \quad
    l = {\bf 0}, \quad
    u = {\bf 1}.
$$
We define these variables in Python using the following code.

In [3]:
# KEY PROBLEM VARIABLES
problem_size = 3
expected_breeding_values = np.array([
    1.0,
    5.0,
    2.0
])
relationship_matrix = np.array([
    [1, 0, 0],
    [0, 5, 0],
    [0, 0, 3]
])
sire_indices = [0]
dam_indices  = [1,2]
lower_bound = np.full((problem_size, 1), 0.0)
upper_bound = np.full((problem_size, 1), 1.0)

# OPTIMIZATION SETUP VARIABLES
lam = 0.5
# define the M so that column i is [1;0] if i is a sire and [0;1] otherwise 
M = np.zeros((2, problem_size))
M[0, sire_indices] = 1
M[1, dam_indices] = 1
# define the right hand side of the constraint Mx = m
m = np.array([[0.5], [0.5]])

### Using Gurobi

To illustrate how the setup changes when uncertainty is added, we will first look at how Gurobi handles the standard problem. The following code returns the same solution as 

In [4]:
# create a model for standard genetic selection
model = gp.Model("standardGS")

# define variable of interest as a continuous 
w = model.addMVar(shape=problem_size, lb=0.0, vtype=GRB.CONTINUOUS, name="w")

# set the objective function
model.setObjective(
    0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values,
GRB.MINIMIZE)

# add sub-to-half constraints
model.addConstr(M @ w == m, name="sum-to-half")
# add weight-bound constraints
model.addConstr(w >= lower_bound, name="lower-bound")
model.addConstr(w <= upper_bound, name="upper-bound")

# solve the problem with Gurobi
model.optimize()
print(f"w = {w.X}")

# save variable for later comparison
std_w = w.X

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 3 columns and 24 nonzeros
Model fingerprint: 0xd96d3c43
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 2e+00]
  QObjective range [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective   

We will define
$$
    \bar{\mu} = \begin{bmatrix} 1 \\ 5 \\ 2 \end{bmatrix},\quad
        \Omega = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 4 & 0 \\ 0 & 0 & \frac{1}{8} \end{bmatrix},
$$
and our other problem variables as before.

In [5]:
omega = np.array([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 1/4]
])

kappa = 0

We then formulate this in Python as follows.

In [6]:
# create a new model for robust genetic selection
model = gp.Model("robustGS")

# define variables of interest as a continuous
w = model.addMVar(shape=problem_size, lb=0.0, vtype=GRB.CONTINUOUS, name="w")
z = model.addVar(lb=0.0, name="z")

# setup the robust objective function
model.setObjective(
    0.5*w@(relationship_matrix@w) - lam*w.transpose()@expected_breeding_values - lam*kappa*z,
GRB.MINIMIZE)

# add quadratic uncertainty constraint
model.addConstr(z**2 <= np.inner(w, omega@w), name="uncertainty")
# add sub-to-half constraints
model.addConstr(M @ w == m, name="sum-to-half")
# add weight-bound constraints~
model.addConstr(w >= lower_bound, name="lower-bound")
model.addConstr(w <= upper_bound, name="upper-bound")

# solve the problem with Gurobi
model.optimize()
print(f"Standard: w = {std_w},\nRobust:   w = {w.X},\n          z = {z.X}.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 4 columns and 24 nonzeros
Model fingerprint: 0x52c6cd50
Model has 3 quadratic objective terms
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [2e-01, 2e+00]
  Objective range  [5e-01, 2e+00]
  QObjective range [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+00]
Presolve removed 21 rows and 1 columns
Presolve time: 0.01s
Presolved: 9 rows, 8 columns, 14 nonzeros
Presolved model has 3 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Resid

Note that even though $\kappa = 0$, we have $z\neq0$. This suggests that a small amount of numerical error has been introduced as a result of asking Gurobi to optimize $z$ as well as $w$. 